In [1]:
import torch
from torch import nn
from torch.nn import functional as F

In [2]:
net = nn.Sequential(nn.Linear(20, 256),
                    nn.ReLU(),
                    nn.Linear(256, 10))

In [3]:
X = torch.rand(2, 20)
net(X)

tensor([[-0.0104,  0.1332, -0.1442,  0.0799, -0.1204, -0.2064,  0.2261,  0.0368,
          0.0533, -0.0725],
        [ 0.0566,  0.0712, -0.1439,  0.1629, -0.1450, -0.2490,  0.2066, -0.0046,
          0.1448, -0.0257]], grad_fn=<AddmmBackward0>)

In [5]:
# 自定义块
class MLP(nn.Module):
    # 用模型参数生命层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 20)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

In [6]:
net = MLP()
net(X)

tensor([[ 0.2567,  0.1547, -0.0342,  0.0029,  0.0423, -0.0616, -0.1228,  0.1050,
         -0.0727, -0.1465,  0.2357, -0.0642, -0.0311,  0.3007,  0.0672,  0.0436,
          0.0659, -0.1650,  0.2552,  0.1052],
        [ 0.1938,  0.1523, -0.0840,  0.0100,  0.1036, -0.0998, -0.0637,  0.1478,
         -0.1403, -0.1701,  0.1307, -0.0432, -0.0368,  0.2316, -0.0150,  0.0471,
          0.0812, -0.0378,  0.2534,  0.0044]], grad_fn=<AddmmBackward0>)

In [9]:
# 顺序快
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是module子类的一个实例，我们把它保存在Module类的成员
            # 变量_modules中。_module的类型是OrderDict
            """
        _modules的主要优点是：在模块的参数初始化过程中，系统知道在_modules字典中查找需要初始化参数的子块。
            """
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderDict 保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

In [10]:
net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[-0.2632, -0.3513,  0.1071, -0.3144, -0.2489, -0.1647, -0.0544,  0.1771,
          0.1435,  0.2231],
        [-0.1647, -0.2454,  0.2032, -0.1838, -0.2431, -0.1272,  0.0017,  0.2266,
          0.0758,  0.1881]], grad_fn=<AddmmBackward0>)

In [11]:
# 在前向传播函数中执行代码

In [13]:
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算地图的随机权重参数，因此在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        x = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

In [14]:
net = FixedHiddenMLP()
net(X)

tensor(0.8899, grad_fn=<SumBackward0>)

In [18]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20,64),
                                 nn.ReLU(),
                                 nn.Linear(64, 32),
                                 nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(0.7869, grad_fn=<SumBackward0>)

In [20]:
# 平行块
class MyModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.net1 = nn.Sequential(nn.Linear(20, 128),
                                  nn.ReLU())
        self.net2 = nn.Sequential(nn.Linear(20, 256),
                                  nn.ReLU())

    def forward(self, X):
        x1 = self.net1(X)
        x2 = self.net2(X)
        x3 = torch.cat((x1, x2), 1)  # 相同行
        Twao = nn.Sequential(nn.Linear(x3.shape[1], 10),
                             nn.ReLU())
        return Twao(x3)

In [23]:
a = torch.arange(9).reshape(3, 3)
b = torch.arange(9).reshape(3, 3)
torch.cat((a, b), dim=1)

tensor([[0, 1, 2, 0, 1, 2],
        [3, 4, 5, 3, 4, 5],
        [6, 7, 8, 6, 7, 8]])